In [1]:
import numpy as np
import tensorflow as tf
import math
from tensorflow.contrib import layers

# conv layer

In [2]:
# image generated
height, width = 3, 3
images = np.random.uniform(size=(5, height, width, 3))
output = tf.contrib.layers.avg_pool2d(images, [3, 3])

# conv layer
output = tf.contrib.layers.convolution2d(images, num_outputs=32, kernel_size=[3, 3])
weights = tf.contrib.framework.get_variables_by_name('weights')[0]
weights_shape = weights.get_shape().as_list()

# image matrix
images = tf.random_uniform((5, height, width, 32), seed=1)

# arg_scope
with tf.contrib.framework.arg_scope([tf.contrib.layers.convolution2d],
                                    normalizer_fn=tf.contrib.layers.batch_norm,
                                    normalizer_params={'decay': 0.9}):
    net = tf.contrib.layers.convolution2d(images, 32, [3 ,3])
    net = tf.contrib.layers.convolution2d(net, 32, [3, 3])
print(len(tf.contrib.framework.get_variables('Conv/BatchNorm')))

0


# fully_connected layer

In [3]:
height, width = 3, 3
inputs = np.random.uniform(size=(5, height, width, 3))
with tf.name_scope('fe'):
    fc = tf.contrib.layers.fully_connected(inputs, 7,
                outputs_collections='outputs', scope='fc') 
output_collected = tf.get_collection('outputs')[0]
print(output_collected)

Tensor("fe/fc/Relu:0", shape=(5, 3, 3, 7), dtype=float64)


# loss 

In [4]:
# absolute difference
predictions = tf.constant([4, 8, 12, 8, 1, 3], shape=(2, 3))
targets = tf.constant([1, 9, 2, -5, -2, 6], shape=(2 ,3))
weight = tf.constant([1.2, 0.0])
loss = tf.contrib.losses.absolute_difference(predictions, targets, weight) 
print(tf.reduce_mean(loss))

# softmax cross entropy
predictions = tf.constant([[10.0, 0.0, 0.0], [0.0, 10.0, 0.0], [0.0, 0.0, 10.0]])
labels = tf.constant([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
loss = tf.contrib.losses.softmax_cross_entropy(predictions, labels)
print(loss)

# softmax with label_smoothing
logits = tf.constant([[100.0, -100.0, -100.0]])
labels = tf.constant(([[1, 0, 0]]))
label_smoothing = 0.1
loss = tf.contrib.losses.softmax_cross_entropy(logits, labels,
                        label_smoothing=label_smoothing)
print(loss)

# sparse softmax cross entropy
logits = tf.constant([[10.0, 0.0, 0.0], [0.0, 10.0, 0.0],[0.0, 0.0, 10.0]])
labels = tf.constant([[0], [1], [2]], dtype=tf.int64)
loss = tf.contrib.losses.sparse_softmax_cross_entropy(logits, labels)
print(loss)

Instructions for updating:
Use tf.losses.absolute_difference instead.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Tensor("Mean:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Tensor("softmax_cross_entropy_loss/value:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Tensor("softmax_cross_entropy_loss_1/value:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.losses.sparse_softmax_cross_e

# feature column

In [5]:
# define data
age = layers.real_valued_column("age")
income = layers.real_valued_column("income")
spending = layers.real_valued_column("spending")
hours_of_work = layers.real_valued_column("hours_of_work")

gender = layers.sparse_column_with_keys(column_name="gender", keys=["female", "male"])
education = layers.sparse_column_with_hash_bucket("education", hash_bucket_size=1000)
age_range = layers.bucketized_column(age, boundaries=[19, 20, 30, 35, 40, 45, 55, 58, 60])

print(age)
print(gender)
print(education)
print(age_range)

_RealValuedColumn(column_name='age', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)
_SparseColumn(column_name='gender', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('female', 'male'), num_oov_buckets=0, vocab_size=2, default_value=-1), combiner='sum', dtype=tf.string)
_SparseColumn(column_name='education', is_integerized=False, bucket_size=1000, lookup_config=None, combiner='sum', dtype=tf.string)
_BucketizedColumn(source_column=_RealValuedColumn(column_name='age', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), boundaries=(19, 20, 30, 35, 40, 45, 55, 58, 60))


# embeddings

In [14]:
indices = [[0, 0], [0, 1], [0, 2], [1, 0], [3, 0], [4, 0], [4, 1]]
ids = [0, 1, -1, -1, 2, 0, 1]
weights = [1.0, 2.0, 1.0, 1.0, 3.0, 0.0, -0.5]
shape = [5, 4]

sparse_ids = tf.SparseTensor(tf.constant(indices, tf.int64),
                            tf.constant(ids, tf.int64),
                            tf.constant(shape, tf.int64))

sparse_weights = tf.SparseTensor(tf.constant(indices, tf.int64),
                                tf.constant(weights, tf.float32),
                                tf.constant(shape, tf.int64))

vocab_size = 4
embed_dim = 4
num_shards = 1

embedding_weights = tf.create_partitioned_variables(
    shape=[vocab_size, embed_dim],
    slicing=[num_shards, 1],
    initializer=tf.truncated_normal_initializer(mean=0.0,
                stddev=1.0/math.sqrt(vocab_size), dtype=tf.float32))

sess = tf.Session()